In [1]:
data_dir = './data/mlboot_dataset/'
model_name = 'e9'
results_dir = './results/'

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import scipy.sparse as sp
import gc

In [2]:
df = pd.read_csv(data_dir + 'preprocessed_new.csv') 
q = pd.read_csv(data_dir + 'sessions.csv')
df = df.merge(q, on='uid', how='left')
del q
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')

df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

mat1 = sp.load_npz(data_dir+'dmat1.npz').tolil()
mat2 = sp.load_npz(data_dir+'dmat2.npz').tolil()
mat3 = sp.load_npz(data_dir+'dmat3.npz').tolil()
print(mat1.shape, mat2.shape, mat3.shape)

(609018, 2053602) (609018, 20275) (609018, 1057788)


In [3]:
df['max_f1'] = mat1.tocsr().max(axis=1).todense()
df['max_f2'] = mat2.tocsr().max(axis=1).todense()
df['max_f3'] = mat3.tocsr().max(axis=1).todense()

train_mat1 = mat1[df_train_index.tolist()]
test_mat1 = mat1[df_test_index.tolist()]
train_mat2 = mat2[df_train_index.tolist()]
test_mat2 = mat2[df_test_index.tolist()]
train_mat3 = mat3[df_train_index.tolist()]
test_mat3 = mat3[df_test_index.tolist()]

limit = 11
mat1 = mat1.tocsc()[:, np.where((train_mat1.getnnz(axis=0) > limit) & (test_mat1.getnnz(axis=0) > 0))[0]].tocsr()
mat2 = mat2.tocsc()[:, np.where((train_mat2.getnnz(axis=0) > limit) & (test_mat2.getnnz(axis=0) > 0))[0]].tocsr()
mat3 = mat3.tocsc()[:, np.where((train_mat3.getnnz(axis=0) > limit) & (test_mat3.getnnz(axis=0) > 0))[0]].tocsr()

In [4]:
print(mat1.shape, mat2.shape, mat3.shape)

(609018, 195734) (609018, 20268) (609018, 9415)


In [5]:
data_svd = pd.DataFrame(np.load(data_dir + 'pca_cat10.npy'), index=df.index)
data_svd.columns = ['svd_description_'+str(i+1) for i in range(10)]
df = pd.concat([df, data_svd], axis=1)    
del data_svd
data_svd = pd.DataFrame(np.load(data_dir + 'apca_dim10.npy'), index=df.index)
data_svd.columns = ['asvd_'+str(i+1) for i in range(10)]
df = pd.concat([df, data_svd], axis=1)    
del data_svd

In [6]:
data_svd = pd.DataFrame(np.load(data_dir + 'bin_pca_dim10.npy'), index=df.index)
data_svd.columns = ['svd_title_'+str(i+1) for i in range(10)]
df = pd.concat([df, data_svd], axis=1)    
del data_svd

In [7]:
train_mat1 = mat1[df_train_index.tolist()]
test_mat1 = mat1[df_test_index.tolist()]
train_mat2 = mat2[df_train_index.tolist()]
test_mat2 = mat2[df_test_index.tolist()]
train_mat3 = mat3[df_train_index.tolist()]
test_mat3 = mat3[df_test_index.tolist()]

del mat1,mat2,mat3
gc.collect()

9

In [8]:
X = df.loc[~df.target.isnull(),:].reset_index(drop=True)
x_te = df.loc[df.target.isnull(),:].reset_index(drop=True)

In [9]:
df.columns

Index(['uid', 'num_times_cat_eq_0', 'num_times_cat_eq_1', 'num_times_cat_eq_2',
       'num_times_cat_eq_3', 'num_times_cat_eq_4', 'num_times_cat_eq_5',
       'records', 'max_days', 'min_days',
       ...
       'svd_title_1', 'svd_title_2', 'svd_title_3', 'svd_title_4',
       'svd_title_5', 'svd_title_6', 'svd_title_7', 'svd_title_8',
       'svd_title_9', 'svd_title_10'],
      dtype='object', length=117)

In [10]:
from sklearn.preprocessing import minmax_scale

In [11]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_baseline_sparse_10folds.npy'))
sample_sub.columns = ['uid','lgbm1']
x_te = x_te.merge(sample_sub, on='uid', how='left')

In [12]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_all_in_focal_loss.npy'))
sample_sub.columns = ['uid','nnet2']
x_te = x_te.merge(sample_sub, on='uid', how='left')

In [13]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_focal_loss_m3.npy'))
sample_sub.columns = ['uid','nnet3']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_focal_loss_m1.npy'))
sample_sub.columns = ['uid','nnet1']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_ftrl_50.npy'))
sample_sub.columns = ['uid','ftrl_50']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_ftrl_70.npy'))
sample_sub.columns = ['uid','ftrl']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_advanced_model.npy'))
sample_sub.columns = ['uid','nnet4']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_advanced_model_catpca.npy'))
sample_sub.columns = ['uid','nnet5']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_advanced_model_3br.npy'))
sample_sub.columns = ['uid','nnet6']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_vw_03perc_099powert_ftrl.npy'))
sample_sub.columns = ['uid','vw']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_vw_07perc.npy'))
sample_sub.columns = ['uid','vw2']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_sgd_091perc.npy'))
sample_sub.columns = ['uid','sgd1']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_tfifg_5folds.npy'))
sample_sub.columns = ['uid','tfidf1']
x_te = x_te.merge(sample_sub, on='uid', how='left')
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_tfifg_svd_5folds.npy'))
sample_sub.columns = ['uid','tfidf2']
x_te = x_te.merge(sample_sub, on='uid', how='left')

In [14]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_advanced_model_3br_wcc.npy'))
sample_sub.columns = ['uid','nnet7']
x_te = x_te.merge(sample_sub, on='uid', how='left')

In [15]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_advanced_model_3br_tanh.npy'))
sample_sub.columns = ['uid','nnet8']
x_te = x_te.merge(sample_sub, on='uid', how='left')

In [16]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_binary_lgbm.npy'))
sample_sub.columns = ['uid','lgbmb']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_advanced_model_3br_v4.npy'))
sample_sub.columns = ['uid','nnet10']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_advanced_model_3br_v5.npy'))
sample_sub.columns = ['uid','nnet11']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_xgb_single.npy'))
sample_sub.columns = ['uid','xgb_single']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_3br_bin_v2.npy'))
sample_sub.columns = ['uid','nnet12']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_advanced_model_3br_bin.npy'))
sample_sub.columns = ['uid','nnet13']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_3br_bin_v4.npy'))
sample_sub.columns = ['uid','nnet14']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_3br_bin_v5.npy'))
sample_sub.columns = ['uid','nnet15']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_sparse_10folds_timp.npy'))
sample_sub.columns = ['uid','timp']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_3br_bin_v8.npy'))
sample_sub.columns = ['uid','nnet16']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_3br_bin_v9.npy'))
sample_sub.columns = ['uid','nnet17']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_we_v2.npy'))
sample_sub.columns = ['uid','nnet18']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_all_in_focal_loss.npy'))
sample_sub.columns = ['uid','nnet19']
x_te = x_te.merge(sample_sub, on='uid', how='left')

In [17]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_fm_svd_3br.npy'))
sample_sub.columns = ['uid','nnet20']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_deep_tree.npy'))
sample_sub.columns = ['uid','deeptree']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_fm_svd_3br_v3.npy'))
sample_sub.columns = ['uid','nnet21']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_nn_we_v5.npy'))
sample_sub.columns = ['uid','nnet22']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_deep_tree1.npy'))
sample_sub.columns = ['uid','dt1']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_deep_tree2.npy'))
sample_sub.columns = ['uid','dt2']
x_te = x_te.merge(sample_sub, on='uid', how='left')

In [18]:
X['lgbm1'] = minmax_scale(np.load(results_dir + 'train_baseline_sparse_10folds.npy'))
X['lgbmb'] = minmax_scale(np.load(results_dir + 'train_binary_lgbm.npy'))
X['nnet2'] = minmax_scale(np.load(results_dir + 'train_all_in_focal_loss.npy'))
X['nnet1'] = minmax_scale(np.load(results_dir + 'train_focal_loss_m1.npy'))
X['nnet3'] = minmax_scale(np.load(results_dir + 'train_focal_loss_m3.npy'))
X['nnet4'] = minmax_scale(np.load(results_dir + 'train_nn_advanced_model.npy'))
X['nnet5'] = minmax_scale(np.load(results_dir + 'train_nn_advanced_model_catpca.npy'))
X['nnet6'] = minmax_scale(np.load(results_dir + 'train_nn_advanced_model_3br.npy'))
X['nnet7'] = minmax_scale(np.load(results_dir + 'train_nn_advanced_model_3br_wcc.npy'))
X['nnet8'] = minmax_scale(np.load(results_dir + 'train_nn_advanced_model_3br_tanh.npy'))
X['ftrl_50']  = minmax_scale(np.load(results_dir + 'train_ftrl_50.npy'))
X['ftrl']  = minmax_scale(np.load(results_dir + 'train_ftrl_70.npy'))
X['nnet10'] = minmax_scale(np.load(results_dir + 'train_nn_advanced_model_3br_v4.npy'))
X['nnet11'] = minmax_scale(np.load(results_dir + 'train_nn_advanced_model_3br_v5.npy'))
X['xgb_single']  = minmax_scale(np.load(results_dir + 'train_xgb_single.npy'))
X['nnet12'] = minmax_scale(np.load(results_dir + 'train_nn_3br_bin_v2.npy'))
X['nnet13'] = minmax_scale(np.load(results_dir + 'train_nn_advanced_model_3br_bin.npy'))
X['vw'] = minmax_scale(np.load(results_dir + 'train_vw_03perc_099powert_ftrl.npy'))
X['vw2'] = minmax_scale(np.load(results_dir + 'train_vw_07perc.npy'))
X['sgd1'] = minmax_scale(np.load(results_dir + 'train_sgd_091perc.npy'))
X['nnet14'] = minmax_scale(np.load(results_dir + 'train_nn_3br_bin_v4.npy'))
X['nnet15'] = minmax_scale(np.load(results_dir + 'train_nn_3br_bin_v5.npy'))
X['timp'] = minmax_scale(np.load(results_dir + 'train_sparse_10folds_timp.npy'))
X['nnet16'] = minmax_scale(np.load(results_dir + 'train_nn_3br_bin_v8.npy'))
X['nnet17'] = minmax_scale(np.load(results_dir + 'train_nn_3br_bin_v9.npy'))
X['nnet18'] = minmax_scale(np.load(results_dir + 'train_nn_we_v2.npy'))
X['tfidf1'] = minmax_scale(np.load(results_dir + 'train_tfifg_5folds.npy'))
X['tfidf2'] = minmax_scale(np.load(results_dir + 'train_tfifg_svd_5folds.npy'))
X['nnet19'] = minmax_scale(np.load(results_dir + 'train_all_in_focal_loss.npy'))
X['nnet20'] = minmax_scale(np.load(results_dir + 'train_fm_svd_3br.npy'))
X['deeptree'] = minmax_scale(np.load(results_dir + 'train_deep_tree.npy'))
X['nnet21'] = minmax_scale(np.load(results_dir + 'train_fm_svd_3br_v3.npy'))
X['nnet22'] = minmax_scale(np.load(results_dir + 'train_nn_we_v5.npy'))
X['dt1'] = minmax_scale(np.load(results_dir + 'train_deep_tree1.npy'))
X['dt2'] = minmax_scale(np.load(results_dir + 'train_deep_tree2.npy'))
#

In [19]:
for f in ['lgbm1','nnet2','nnet1','nnet3','nnet4','nnet5','ftrl',
          'nnet6','nnet8','ftrl_50','nnet11','xgb_single','lgbmb',
          'nnet12','nnet13','nnet14','nnet15','nnet18','tfidf1','tfidf2','nnet19','nnet20']:
    print(f,roc_auc_score(X.target, X[f]))

lgbm1 0.682562603224
nnet2 0.681848937554
nnet1 0.623698784984
nnet3 0.574520607881
nnet4 0.647936920552
nnet5 0.648281500429
ftrl 0.622924063126
nnet6 0.658849449887
nnet8 0.657409030233
ftrl_50 0.623440994459
nnet11 0.660053076044
xgb_single 0.660175309563
lgbmb 0.66792997872
nnet12 0.67274153339
nnet13 0.67264163497
nnet14 0.671105467501
nnet15 0.678910109891
nnet18 0.677717274239
tfidf1 0.679597805043
tfidf2 0.682393866377
nnet19 0.681848937554
nnet20 0.682987673047


In [20]:
X[['nnet12','nnet13','tfidf2','vw2','sgd1','nnet18','nnet17','nnet19']].corr()

,nnet12,nnet13,tfidf2,vw2,sgd1,nnet18,nnet17,nnet19
nnet12,1.000000,0.815829,0.671611,0.769820,0.780070,0.795601,0.814261,0.713636
nnet13,0.815829,1.000000,0.780991,0.845585,0.806378,0.703029,0.686586,0.769341
tfidf2,0.671611,0.780991,1.000000,0.740278,0.707507,0.690532,0.672900,0.871544
vw2,0.769820,0.845585,0.740278,1.000000,0.918504,0.725528,0.699212,0.756955
sgd1,0.780070,0.806378,0.707507,0.918504,1.000000,0.727218,0.727685,0.735116
nnet18,0.795601,0.703029,0.690532,0.725528,0.727218,1.000000,0.813697,0.751477
nnet17,0.814261,0.686586,0.672900,0.699212,0.727685,0.813697,1.000000,0.730332
nnet19,0.713636,0.769341,0.871544,0.756955,0.735116,0.751477,0.730332,1.000000


In [21]:
x_te[['nnet12','nnet13','tfidf2','vw2','sgd1','nnet18','nnet17','nnet19']].corr()

,nnet12,nnet13,tfidf2,vw2,sgd1,nnet18,nnet17,nnet19
nnet12,1.000000,0.872521,0.690019,0.795288,0.776215,0.827933,0.833681,0.720283
nnet13,0.872521,1.000000,0.785687,0.845255,0.793905,0.740004,0.721948,0.781601
tfidf2,0.690019,0.785687,1.000000,0.703277,0.655306,0.715536,0.700106,0.861969
vw2,0.795288,0.845255,0.703277,1.000000,0.922127,0.726212,0.718369,0.729844
sgd1,0.776215,0.793905,0.655306,0.922127,1.000000,0.685385,0.709385,0.691483
nnet18,0.827933,0.740004,0.715536,0.726212,0.685385,1.000000,0.858265,0.758730
nnet17,0.833681,0.721948,0.700106,0.718369,0.709385,0.858265,1.000000,0.739510
nnet19,0.720283,0.781601,0.861969,0.729844,0.691483,0.758730,0.739510,1.000000


In [19]:
train_mat = sp.hstack([train_mat1,train_mat2,train_mat3]).tocsr()
test_mat = sp.hstack([test_mat1,test_mat2,test_mat3]).tocsr()

In [20]:
mat = test_mat.sum(axis=0)
ixs = np.asarray(mat)[0].argsort()[-5000:][::-1]
train_mat = train_mat[:,ixs]
test_mat = test_mat[:,ixs]

In [21]:
mat_pca = np.load(data_dir + 'svd_tfidf300.npy')
train_mat_pcat = mat_pca[df_train_index.tolist()]
test_mat_pcat = mat_pca[df_test_index.tolist()]

In [23]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_deep_tree3.npy'))
sample_sub.columns = ['uid','dt3']
x_te = x_te.merge(sample_sub, on='uid', how='left')
X['dt3'] = minmax_scale(np.load(results_dir + 'train_deep_tree3.npy'))

In [35]:
model_name = 'lgb_e9'
import xgboost as xgb
import scipy.sparse as sp
from sklearn.feature_selection import SelectPercentile
def save_submit(model_name, folds, y_pred):
    global x_te
    sub = x_te[['uid','target']].copy()
    sub['target'] = y_pred
    sub.columns = ['cuid','target']
    sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
    sample_sub = sample_sub.merge(sub, on='cuid', how='left')
    sample_sub[['target']].to_csv(results_dir + model_name + '_' + str(folds) + 'folds.csv', header=False, index=False)
    del sub,sample_sub
    gc.collect()
    
def mean_encode_test(df, y, test,k,column):
    mean_0 = np.zeros((test.shape[0],1))
    df['target'] = y
    m0 = np.mean(y)  
    y0s = df[['target',column]].groupby(column).agg(np.mean).reset_index()
    y0s.columns = [column,'target_mean']
    vc = df[column].value_counts().reset_index()
    vc.columns = [column,'counts']
    test = test.merge(y0s, on = column,how= 'left').merge(vc, on = column,how= 'left')
    test['mean_target'] = (test.target_mean * test.counts + k * m0)/(test.counts + k)
    mean_0 = np.array(test['mean_target']).reshape(-1,1)
    return mean_0    

def mean_encode_self(df, y, kf, k, column):
    mean_0 = np.zeros((y.shape[0],1))
    df['target'] = y
    m0 = np.mean(y)
    for dev_index, val_index in kf: 
        dev_X, val_X = df.iloc[dev_index,:], df.iloc[val_index,:]
        y0s = dev_X[['target',column]].groupby(column).agg(np.mean).reset_index()
        y0s.columns = [column,'target_mean']
        vc = dev_X[column].value_counts().reset_index()
        vc.columns = [column,'counts']
        val_X = val_X.merge(y0s, on = column,how= 'left').merge(vc, on = column,how= 'left')
        val_X['mean_target'] = (val_X.target_mean * val_X.counts + k * m0)/(val_X.counts + k)
        mean_0[val_index,:] = np.array(val_X['mean_target']).reshape(-1,1)       
    return mean_0

def make_agg_features(X, train_index, test_index, test_data):
    te_cols = ['most_freq_cat']
    kf = KFold(n_splits = 5, random_state=2018, shuffle=True)
    for c in te_cols:
        X.loc[test_index,c + '_te'] = mean_encode_test(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), X.loc[test_index,:].copy(), 10.0, c)
        test_data.loc[:,c + '_te'] = mean_encode_test(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), test_data.copy(), 10.0, c)
        X.loc[train_index,c + '_te'] = mean_encode_self(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), kf.split(X.loc[train_index,:]), 10.0, c)
    return X.loc[train_index,:], X.loc[test_index,:], test_data
    
train_cols = ['sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
              'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
              'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
              'num_times_cat_eq_0', 'num_times_cat_eq_1', 'num_times_cat_eq_2',
              'num_times_cat_eq_3', 'num_times_cat_eq_4', 'num_times_cat_eq_5',
              'records', 'max_days', 'min_days', 'sum_values_f1_max',
              'num_keys_f1_max', 'sum_values_f2_max', 'num_keys_f2_max',
              'sum_values_f3_max', 'num_keys_f3_max', 'sum_values_f1_mean',
              'num_keys_f1_mean', 'sum_values_f2_mean', 'num_keys_f2_mean',
              'sum_values_f3_mean', 'num_keys_f3_mean', 'max_day_cntr',
              'mean_day_cntr', 'nuniq_keys_f1', 'nuniq_keys_f1.1',
              'nuniq_keys_f1.2', 'sumval_keys_f1', 'sumval_keys_f1.1',
              'sumval_keys_f1.2', 'most_freq_cat_te', 'diff_num_cats', 'unique_days','max_f1','max_f2','max_f3',
              'svd_description_1','svd_description_2','svd_description_3','svd_description_4','svd_description_5',
              'svd_description_6','svd_description_7','svd_description_8','svd_description_9','svd_description_10',
              'nnet10','nnet11','xgb_single','nnet12','nnet13','lgbm1','dt2','dt1',
              'nnet3','nnet1','nnet14','nnet15','timp','nnet19','nnet20','nnet21','nnet22',
              'ftrl','vw2','sgd1','nnet16','nnet17','nnet18','tfidf2','tfidf1','deeptree',
              'vw','ftrl_50',
              'nnet4','nnet5','nnet6','nnet7',
              'nnet8','lgbmb','most_freq_cat_te'] + ['svd_title_'+str(i+1) for i in range(10)] + ['asvd_'+str(i+1) for i in range(10)]


#,'most_freq_cat_te','nnet4','nnet5','nnet6','nnet7','nnet10','nnet11','nnet8',

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 16,
    'max_depth' : 4,
    #'min_data' : 30,
    'lambda_l1' : 1.5,
    'lambda_l2' : 30.2,
    'bagging_freq' : 2,
    'bagging_fraction' : 0.8,
    'is_unbalance': True,
    'learning_rate': 0.005,
    'feature_fraction': 0.5,
    'min_data_in_leaf':100,
    'verbose': 0
}

kf = KFold(n_splits=10, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

train_mat = sp.hstack([train_mat1,train_mat2,train_mat3]).tocsr().astype(np.bool).astype(np.float32)
test_mat = sp.hstack([test_mat1,test_mat2,test_mat3]).tocsr().astype(np.bool).astype(np.float32)

for train_index,test_index in kf.split(X):
    print('fold', ifold)
       
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va,X_te = make_agg_features(X,train_index,test_index,x_te)
    X_tr = X_tr[train_cols]
    X_va = X_va[train_cols]
    X_te = X_te[train_cols]
    
    yy = y_tr
    ssp = SelectPercentile(percentile=0.1)  
    ssp.fit(train_mat[train_index], yy)
    sp_train_mat = ssp.transform(train_mat[train_index])
    sp_val_mat = ssp.transform(train_mat[test_index])
    sp_test_mat = ssp.transform(test_mat)   
    
    print('prepare train')
    X_tr = sp.hstack([     X_tr, sp_train_mat   ]).tocsr() #, train_mat_pcat[train_index] 
    #print(X_tr.shape)
    #print('prepare valid')
    X_va = sp.hstack([        X_va, sp_val_mat    ]).tocsr()     #, train_mat_pcat[test_index]
    #print('prepare test')
    X_te = sp.hstack([       X_te, sp_test_mat    ]).tocsr()      #, test_mat_pcat

    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=[tr_data,va_data],
                      num_boost_round=8000,
                      early_stopping_rounds=300,
                      verbose_eval=100)
    
    yhat = model.predict(X_va, model.best_iteration)
    scores.append(roc_auc_score(y_va,yhat))
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    
    ytst = model.predict(X_te, model.best_iteration)
    print(ytst)
    y_pred += minmax_scale(ytst)*0.1
    
    del X_tr,X_va,tr_data,va_data, sp_train_mat, sp_val_mat, sp_test_mat
    gc.collect()    
    
    save_submit('lgb_q', ifold, y_pred)

    ifold += 1     
print(scores)
print(np.mean(scores), np.std(scores))    

np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

fold 0
prepare train
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.697703	valid_1's auc: 0.70875
[200]	training's auc: 0.698523	valid_1's auc: 0.708756
[300]	training's auc: 0.699408	valid_1's auc: 0.708692
Early stopping, best iteration is:
[7]	training's auc: 0.69561	valid_1's auc: 0.709791
0 0.709791259567
prepare test
[ 0.5072458   0.50363608  0.50170444 ...,  0.50263838  0.50170444
  0.49307455]
fold 1
prepare train
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.699895	valid_1's auc: 0.68808
[200]	training's auc: 0.700848	valid_1's auc: 0.6884
[300]	training's auc: 0.701732	valid_1's auc: 0.688419
[400]	training's auc: 0.702636	valid_1's auc: 0.688527
[500]	training's auc: 0.70357	valid_1's auc: 0.688615
[600]	training's auc: 0.704543	valid_1's auc: 0.688672
[700]	training's auc: 0.705597	valid_1's auc: 0.688744
[800]	training's auc: 0.706648	valid_1's auc: 0.688825
[900]	training's auc: 0.707607	vali

,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.925506
1,ac4b8244f3ae82df511b002257473c11,0.536310
2,483d8b91e49522c8a5bbe37f3872c749,0.757470
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.377797
4,fdbfba9842ff0bf86d600eb334c7c42b,0.381895


In [28]:
print(scores)
print(np.mean(scores), np.std(scores))

[0.70935540026201749, 0.6890247976060333, 0.68378549419421653, 0.70274745749668643, 0.69224803818126024, 0.6952706514919984, 0.69102802236127214, 0.70608650004544038, 0.69540192462882777, 0.69774039503128993]
0.69626886813 0.00751979706633


In [ ]:
[0.70919731520431251, 0.6891197652556077, 0.68374056085963286, 0.70273171958127179, 0.69249429319641487, 
 0.69528635958606888, 0.69101162136385164, 0.70591099061834595, 0.69552760688284754, 0.69797274585547964] 
0.69629929784 0.00745819458099

In [27]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_xgb_e7.npy'))
sample_sub.columns = ['uid','xgbe7']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_lgb_e8.npy'))
sample_sub.columns = ['uid','lgbe8']
x_te = x_te.merge(sample_sub, on='uid', how='left')
x_te[['xgbe7','lgbe8']].corr()

,xgbe7,lgbe8
xgbe7,1.000000,0.996499
lgbe8,0.996499,1.000000


In [29]:
model_name = 'blend'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = (x_te.xgbe7 + x_te.lgbe8 + x_te.lgbe7 + x_te.lgbf7)*0.25
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

isnull? False


,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.900125
1,ac4b8244f3ae82df511b002257473c11,0.509793
2,483d8b91e49522c8a5bbe37f3872c749,0.711866
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.377108
4,fdbfba9842ff0bf86d600eb334c7c42b,0.389375


In [28]:
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_lgb_e7.npy'))
sample_sub.columns = ['uid','lgbe7']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_lgb_f7.npy'))
sample_sub.columns = ['uid','lgbf7']
x_te = x_te.merge(sample_sub, on='uid', how='left')


x_te[['xgbe7','lgbe8','lgbe7','lgbf7']].corr()


,xgbe7,lgbe8,lgbe7,lgbf7
xgbe7,1.000000,0.996499,0.991828,0.997217
lgbe8,0.996499,1.000000,0.992490,0.998762
lgbe7,0.991828,0.992490,1.000000,0.994776
lgbf7,0.997217,0.998762,0.994776,1.000000


In [ ]:
[0.70584483206119242, 0.68852257053257726, 0.68120543644470821, 0.69943609670565343, 0.68975482586128489, 
 0.69245894848299694, 0.68597921405405382, 0.70357452572825019, 0.69242992054470409, 0.69611483413886988]
0.693532120455 0.00738040471586

In [24]:
model_name = 'lgb_b2'
import xgboost as xgb
import scipy.sparse as sp
from sklearn.feature_selection import SelectPercentile
def save_submit(model_name, folds, y_pred):
    global x_te
    sub = x_te[['uid','target']].copy()
    sub['target'] = y_pred
    sub.columns = ['cuid','target']
    sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
    sample_sub = sample_sub.merge(sub, on='cuid', how='left')
    sample_sub[['target']].to_csv(results_dir + model_name + '_' + str(folds) + 'folds.csv', header=False, index=False)
    del sub,sample_sub
    gc.collect()
    
def mean_encode_test(df, y, test,k,column):
    mean_0 = np.zeros((test.shape[0],1))
    df['target'] = y
    m0 = np.mean(y)  
    y0s = df[['target',column]].groupby(column).agg(np.mean).reset_index()
    y0s.columns = [column,'target_mean']
    vc = df[column].value_counts().reset_index()
    vc.columns = [column,'counts']
    test = test.merge(y0s, on = column,how= 'left').merge(vc, on = column,how= 'left')
    test['mean_target'] = (test.target_mean * test.counts + k * m0)/(test.counts + k)
    mean_0 = np.array(test['mean_target']).reshape(-1,1)
    return mean_0    

def mean_encode_self(df, y, kf, k, column):
    mean_0 = np.zeros((y.shape[0],1))
    df['target'] = y
    m0 = np.mean(y)
    for dev_index, val_index in kf: 
        dev_X, val_X = df.iloc[dev_index,:], df.iloc[val_index,:]
        y0s = dev_X[['target',column]].groupby(column).agg(np.mean).reset_index()
        y0s.columns = [column,'target_mean']
        vc = dev_X[column].value_counts().reset_index()
        vc.columns = [column,'counts']
        val_X = val_X.merge(y0s, on = column,how= 'left').merge(vc, on = column,how= 'left')
        val_X['mean_target'] = (val_X.target_mean * val_X.counts + k * m0)/(val_X.counts + k)
        mean_0[val_index,:] = np.array(val_X['mean_target']).reshape(-1,1)       
    return mean_0

def make_agg_features(X, train_index, test_index, test_data):
    te_cols = ['most_freq_cat']
    kf = KFold(n_splits = 5, random_state=2018, shuffle=True)
    for c in te_cols:
        X.loc[test_index,c + '_te'] = mean_encode_test(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), X.loc[test_index,:].copy(), 10.0, c)
        test_data.loc[:,c + '_te'] = mean_encode_test(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), test_data.copy(), 10.0, c)
        X.loc[train_index,c + '_te'] = mean_encode_self(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), kf.split(X.loc[train_index,:]), 10.0, c)
    return X.loc[train_index,:], X.loc[test_index,:], test_data
    
train_cols = ['sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
              'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
              'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
              'num_times_cat_eq_0', 'num_times_cat_eq_1', 'num_times_cat_eq_2',
              'num_times_cat_eq_3', 'num_times_cat_eq_4', 'num_times_cat_eq_5',
              'records', 'max_days', 'min_days', 'sum_values_f1_max',
              'num_keys_f1_max', 'sum_values_f2_max', 'num_keys_f2_max',
              'sum_values_f3_max', 'num_keys_f3_max', 'sum_values_f1_mean',
              'num_keys_f1_mean', 'sum_values_f2_mean', 'num_keys_f2_mean',
              'sum_values_f3_mean', 'num_keys_f3_mean', 'max_day_cntr',
              'mean_day_cntr', 'nuniq_keys_f1', 'nuniq_keys_f1.1',
              'nuniq_keys_f1.2', 'sumval_keys_f1', 'sumval_keys_f1.1',
              'sumval_keys_f1.2', 'most_freq_cat_te', 'diff_num_cats', 'unique_days','max_f1','max_f2','max_f3',
              'svd_description_1','svd_description_2','svd_description_3','svd_description_4','svd_description_5',
              'svd_description_6','svd_description_7','svd_description_8','svd_description_9','svd_description_10',
              'nnet4','nnet5','nnet6','nnet7','nnet10','nnet11','xgb_single','nnet12','nnet13','lgbm1','nnet3','nnet1',
              'nnet8','lgbmb','most_freq_cat_te'] + ['svd_title_'+str(i+1) for i in range(10)]

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 16,
    'max_depth' : 4,
    #'min_data' : 100,
    #'lambda_l2' : 15.5,
    'min_sum_hessian_in_leaf' : 0.2,
    'lambda_l1' : 6.2,
    'is_unbalance': True,
    'learning_rate': 0.001,
    'feature_fraction': 0.7,
    'verbose': 0
}

kf = KFold(n_splits=10, shuffle=True, random_state=13)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

train_mat = sp.hstack([train_mat1,train_mat2,train_mat3]).tocsr()
test_mat = sp.hstack([test_mat1,test_mat2,test_mat3]).tocsr()
mat_spca = np.load(data_dir + 'spca_dim100.npy')
train_mat_spca = mat_spca[df_train_index.tolist()]
test_mat_spca = mat_spca[df_test_index.tolist()]
del mat_spca

for train_index,test_index in kf.split(X):
    print('fold', ifold)
       
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va,X_te = make_agg_features(X,train_index,test_index,x_te)
    X_tr = X_tr[train_cols]
    X_va = X_va[train_cols]
    X_te = X_te[train_cols]
    
    yy = y_tr
    ssp = SelectPercentile(percentile=0.1)  
    ssp.fit(train_mat[train_index], yy)
    sp_train_mat = ssp.transform(train_mat[train_index])
    sp_val_mat = ssp.transform(train_mat[test_index])
    sp_test_mat = ssp.transform(test_mat)   
    
    print('prepare train')
    X_tr = sp.hstack([
        X_tr, sp_train_mat, train_mat_spca[train_index]
    ]).tocsr()
    print(X_tr.shape)
    print('prepare valid')
    X_va = sp.hstack([
        X_va, sp_val_mat, train_mat_spca[test_index]
    ]).tocsr()    
    print('prepare test')
    X_te = sp.hstack([
        X_te, sp_test_mat, test_mat_spca
    ]).tocsr()     

    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=[tr_data,va_data],
                      num_boost_round=8000,
                      early_stopping_rounds=300,
                      verbose_eval=100)
    
    yhat = model.predict(X_va, model.best_iteration)
    scores.append(roc_auc_score(y_va,yhat))
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    
    ytst = model.predict(X_te, model.best_iteration)
    y_pred += ytst*0.1
    
    del X_tr,X_va,tr_data,va_data, sp_train_mat, sp_val_mat, sp_test_mat
    gc.collect()    
    
    save_submit('lgb_q', ifold, y_pred)

    ifold += 1     
print(scores)
print(np.mean(scores), np.std(scores))    

np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

fold 0
prepare train
(385194, 410)
prepare valid
prepare test
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.688782	valid_1's auc: 0.680434
[200]	training's auc: 0.689369	valid_1's auc: 0.680718
[300]	training's auc: 0.689748	valid_1's auc: 0.681187
[400]	training's auc: 0.690082	valid_1's auc: 0.681541
[500]	training's auc: 0.690511	valid_1's auc: 0.681837
[600]	training's auc: 0.690828	valid_1's auc: 0.682075
[700]	training's auc: 0.691088	valid_1's auc: 0.682264
[800]	training's auc: 0.691347	valid_1's auc: 0.682386
[900]	training's auc: 0.691583	valid_1's auc: 0.682475
[1000]	training's auc: 0.691845	valid_1's auc: 0.682563
[1100]	training's auc: 0.692089	valid_1's auc: 0.682652
[1200]	training's auc: 0.692325	valid_1's auc: 0.68269
[1300]	training's auc: 0.692538	valid_1's auc: 0.682712
[1400]	training's auc: 0.692752	valid_1's auc: 0.682727
[1500]	training's auc: 0.692951	valid_1's auc: 0.68275
[1600]	training's auc: 0.693164	valid_1's auc:

[2500]	training's auc: 0.694809	valid_1's auc: 0.685585
[2600]	training's auc: 0.695072	valid_1's auc: 0.685611
[2700]	training's auc: 0.695337	valid_1's auc: 0.685642
[2800]	training's auc: 0.6956	valid_1's auc: 0.685645
[2900]	training's auc: 0.695853	valid_1's auc: 0.685642
[3000]	training's auc: 0.696117	valid_1's auc: 0.685645
[3100]	training's auc: 0.696391	valid_1's auc: 0.685631
[3200]	training's auc: 0.696669	valid_1's auc: 0.685628
Early stopping, best iteration is:
[2959]	training's auc: 0.696009	valid_1's auc: 0.68565
4 0.685650472164
prepare test
fold 5
prepare train
(385195, 410)
prepare valid
prepare test
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.68827	valid_1's auc: 0.68582
[200]	training's auc: 0.688599	valid_1's auc: 0.686177
[300]	training's auc: 0.689037	valid_1's auc: 0.686457
[400]	training's auc: 0.689382	valid_1's auc: 0.686671
[500]	training's auc: 0.689775	valid_1's auc: 0.68678
[600]	training's auc: 0.690088	valid_

,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.727645
1,ac4b8244f3ae82df511b002257473c11,0.578061
2,483d8b91e49522c8a5bbe37f3872c749,0.673377
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.476768
4,fdbfba9842ff0bf86d600eb334c7c42b,0.444018


In [25]:
sample_sub.head()

,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.727645
1,ac4b8244f3ae82df511b002257473c11,0.578061
2,483d8b91e49522c8a5bbe37f3872c749,0.673377
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.476768
4,fdbfba9842ff0bf86d600eb334c7c42b,0.444018


In [26]:
X = df.loc[~df.target.isnull(),:].reset_index(drop=True)
x_te = df.loc[df.target.isnull(),:].reset_index(drop=True)

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_xgb_b1.npy'))
sample_sub.columns = ['uid','xgb_sess4']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_lgb_b1.npy'))
sample_sub.columns = ['uid','lgb_sess6']
x_te = x_te.merge(sample_sub, on='uid', how='left')

sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_lgb_b2.npy'))
sample_sub.columns = ['uid','lgb_sess7']
x_te = x_te.merge(sample_sub, on='uid', how='left')

In [27]:
X['xgb_sess4'] = minmax_scale(np.load(results_dir + 'train_xgb_b1.npy'))
X['lgb_sess6'] = minmax_scale(np.load(results_dir + 'train_lgb_b1.npy'))
X['lgb_sess7'] = minmax_scale(np.load(results_dir + 'train_lgb_b2.npy'))

In [35]:
meta = ['nnet12','nnet13','lgbm1','lgbmb','nnet1','nnet3','xgb_single'] #
for f in meta:
    print(f,roc_auc_score(X.target, X[f]))

nnet12 0.67274153339
nnet13 0.67264163497
lgbm1 0.682562603224
lgbmb 0.66792997872
nnet1 0.623698784984
nnet3 0.574520607881
xgb_single 0.660175309563


In [36]:
X[meta].corr()

,nnet12,nnet13,lgbm1,lgbmb,nnet1,nnet3,xgb_single
nnet12,1.000000,0.815829,0.736898,0.844497,0.612611,0.445006,0.825464
nnet13,0.815829,1.000000,0.853592,0.710129,0.463131,0.288266,0.687111
lgbm1,0.736898,0.853592,1.000000,0.752149,0.494446,0.295207,0.698921
lgbmb,0.844497,0.710129,0.752149,1.000000,0.668780,0.501951,0.907113
nnet1,0.612611,0.463131,0.494446,0.668780,1.000000,0.487426,0.661838
nnet3,0.445006,0.288266,0.295207,0.501951,0.487426,1.000000,0.490909
xgb_single,0.825464,0.687111,0.698921,0.907113,0.661838,0.490909,1.000000


In [37]:
x_te[meta].corr()

,nnet12,nnet13,lgbm1,lgbmb,nnet1,nnet3,xgb_single
nnet12,1.000000,0.872521,0.793141,0.837708,0.643211,0.436315,0.821308
nnet13,0.872521,1.000000,0.871866,0.713111,0.511538,0.316164,0.681750
lgbm1,0.793141,0.871866,1.000000,0.778259,0.536187,0.342257,0.726940
lgbmb,0.837708,0.713111,0.778259,1.000000,0.641589,0.466649,0.933161
nnet1,0.643211,0.511538,0.536187,0.641589,1.000000,0.488963,0.639809
nnet3,0.436315,0.316164,0.342257,0.466649,0.488963,1.000000,0.456628
xgb_single,0.821308,0.681750,0.726940,0.933161,0.639809,0.456628,1.000000


In [33]:
import xgboost as xgb
import scipy.sparse as sp
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectPercentile
def save_submit(model_name, folds, y_pred):
    global x_te
    sub = x_te[['uid','target']].copy()
    sub['target'] = y_pred
    sub.columns = ['cuid','target']
    sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
    sample_sub = sample_sub.merge(sub, on='cuid', how='left')
    sample_sub[['target']].to_csv(results_dir + model_name + '_' + str(folds) + 'folds.csv', header=False, index=False)
    del sub,sample_sub
    gc.collect()
    
def mean_encode_test(df, y, test,k,column):
    mean_0 = np.zeros((test.shape[0],1))
    df['target'] = y
    m0 = np.mean(y)  
    y0s = df[['target',column]].groupby(column).agg(np.mean).reset_index()
    y0s.columns = [column,'target_mean']
    vc = df[column].value_counts().reset_index()
    vc.columns = [column,'counts']
    test = test.merge(y0s, on = column,how= 'left').merge(vc, on = column,how= 'left')
    test['mean_target'] = (test.target_mean * test.counts + k * m0)/(test.counts + k)
    mean_0 = np.array(test['mean_target']).reshape(-1,1)
    return mean_0    

def mean_encode_self(df, y, kf, k, column):
    mean_0 = np.zeros((y.shape[0],1))
    df['target'] = y
    m0 = np.mean(y)
    for dev_index, val_index in kf: 
        dev_X, val_X = df.iloc[dev_index,:], df.iloc[val_index,:]
        y0s = dev_X[['target',column]].groupby(column).agg(np.mean).reset_index()
        y0s.columns = [column,'target_mean']
        vc = dev_X[column].value_counts().reset_index()
        vc.columns = [column,'counts']
        val_X = val_X.merge(y0s, on = column,how= 'left').merge(vc, on = column,how= 'left')
        val_X['mean_target'] = (val_X.target_mean * val_X.counts + k * m0)/(val_X.counts + k)
        mean_0[val_index,:] = np.array(val_X['mean_target']).reshape(-1,1)       
    return mean_0

def make_agg_features(X, train_index, test_index, test_data):
    te_cols = ['most_freq_cat']
    kf = KFold(n_splits = 5, random_state=2018, shuffle=True)
    for c in te_cols:
        X.loc[test_index,c + '_te'] = mean_encode_test(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), X.loc[test_index,:].copy(), 10.0, c)
        test_data.loc[:,c + '_te'] = mean_encode_test(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), test_data.copy(), 10.0, c)
        X.loc[train_index,c + '_te'] = mean_encode_self(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), kf.split(X.loc[train_index,:]), 10.0, c)
    return X.loc[train_index,:], X.loc[test_index,:], test_data
    
train_cols = meta

# Train the model
parameters = {
    'booster' : 'gbtree',
    'n_estimators':20000,
    'max_depth':4,
    'objective':"binary:logistic",
    'eval_metric':'auc',
    'learning_rate':0.005, 
    'subsample':.6,
    'min_child_weight':10,
    'colsample_bytree':.6,
    'scale_pos_weight': 19,
    'gamma':1,
    #'reg_alpha':1,
    'reg_lambda':1.3,
}

kf = KFold(n_splits=10, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):
    print('fold', ifold)
       
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va,X_te = make_agg_features(X,train_index,test_index,x_te)

    
    X_tr = X_tr[train_cols].fillna(0).values
    X_va = X_va[train_cols].fillna(0).values
    X_te = X_te[train_cols].fillna(0).values
    
       
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_va = scaler.transform(X_va)
    X_te = scaler.transform(X_te)
    del scaler

    y_hat_med = pd.DataFrame(X_va).mean(axis=1)
    #print(ifold,'median:',roc_auc_score(y_va,y_hat_med))
    
    
    # Create the LightGBM data containers
    model = Ridge(alpha=20)
    model.fit(X_tr,y_tr)
    
    yhat = model.predict(X_va)
    scores.append(roc_auc_score(y_va,yhat))
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    
    #
    ytst = pd.DataFrame(X_te).mean(axis=1)
    ytst = model.predict(X_te)
    print(minmax_scale(ytst))
    y_pred += minmax_scale(ytst)*0.1
    
    del X_tr,X_va,X_te
    gc.collect()    
    
    save_submit('xgb_q', ifold, y_pred)

    ifold += 1    

fold 0
0 0.698668673061
prepare test
[ 0.14689715  0.14794698  0.10039477 ...,  0.07865642  0.0795201
  0.04324515]
fold 1
1 0.680282222997
prepare test
[ 0.14906741  0.14968694  0.10206368 ...,  0.08114234  0.08153055
  0.04605674]
fold 2
2 0.672018064269
prepare test
[ 0.14804478  0.14921921  0.10076937 ...,  0.07930363  0.07985425
  0.04391066]
fold 3
3 0.688242992587
prepare test
[ 0.14820545  0.1487872   0.10010541 ...,  0.07902909  0.07945207
  0.04392936]
fold 4
4 0.681608416947
prepare test
[ 0.14675651  0.14805475  0.10015005 ...,  0.07874264  0.07952918
  0.0439385 ]
fold 5
5 0.683953307708
prepare test
[ 0.14593103  0.14723824  0.09932246 ...,  0.07791339  0.07879705
  0.04330436]
fold 6
6 0.678530035578
prepare test
[ 0.15262749  0.15368207  0.10530478 ...,  0.08421273  0.08437358
  0.04908429]
fold 7
7 0.694977085688
prepare test
[ 0.14813504  0.14974     0.10206602 ...,  0.08021212  0.08106084
  0.04498688]
fold 8
8 0.683650882628
prepare test
[ 0.14763976  0.14892431  0.

In [34]:
print(scores)
print(np.mean(scores), np.std(scores))
roc_auc_score(X.target.values, y_oof.target.values)

[0.69866867306089697, 0.68028222299680186, 0.6720180642689908, 0.6882429925869018, 0.6816084169472465, 0.68395330770756191, 0.67853003557793867, 0.69497708568816041, 0.68365088262772289, 0.69192788633314306]
0.68538595678 0.00768417125345


0.68522457782906487

In [28]:
model_name = 'ridge_e1'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

isnull? False


,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.246518
1,ac4b8244f3ae82df511b002257473c11,0.106445
2,483d8b91e49522c8a5bbe37f3872c749,0.165956
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.074222
4,fdbfba9842ff0bf86d600eb334c7c42b,0.064321


In [44]:
s = pd.DataFrame()
s['s7463'] = minmax_scale(pd.read_csv(results_dir + 'ridge_b1.csv', header=None, names=['v']).v.values)
s['s7412'] = minmax_scale(pd.read_csv(results_dir + 'baseline_sparse_10folds.csv', header=None, names=['v']).v.values)

In [45]:
s.head()

,s7463,s7412
0,0.811052,0.184876
1,0.570673,0.096343
2,0.726611,0.151491
3,0.429136,0.055863
4,0.389015,0.051491


In [40]:
len(sample_sub)

181024

In [39]:
model_name = 'ridge_st'
import xgboost as xgb
import scipy.sparse as sp
from sklearn.feature_selection import SelectPercentile
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge

def make_agg_features(X, train_index, test_index, test_data):
    return X.loc[train_index,:], X.loc[test_index,:], test_data
    
train_cols = ['records', 'max_days', 'min_days', 
              'nnet10','nnet11','xgb_single','nnet12','nnet13','lgbm1','dt2','dt1','dt3',
              'nnet3','nnet1','nnet14','nnet15','timp','nnet19','nnet20','nnet21','nnet22',
              'ftrl','vw2','sgd1','nnet16','nnet17','nnet18','tfidf2','tfidf1','deeptree',
              'vw','ftrl_50',
              'nnet4','nnet5','nnet6','nnet7',
              'nnet8','lgbmb'] + ['svd_title_'+str(i+1) for i in range(3)] + ['asvd_'+str(i+1) for i in range(3)]

kf = KFold(n_splits=10, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):
    print('fold', ifold)
       
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va,X_te = make_agg_features(X,train_index,test_index,x_te)
    X_tr = X_tr[train_cols].fillna(0).values
    X_va = X_va[train_cols].fillna(0).values
    X_te = X_te[train_cols].fillna(0).values
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    X_tr = scaler.fit_transform(X_tr)
    X_va = scaler.transform(X_va)
    X_te = scaler.transform(X_te)
    
    model = Ridge()
    model.fit(X_tr,y_tr)
    yhat = model.predict(X_va)
    scores.append(roc_auc_score(y_va,yhat))
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    
    ytst = model.predict(X_te)
    print(ytst)
    y_pred += minmax_scale(ytst)*0.1
    
    del X_tr,X_va,X_te
    gc.collect()    
    
    ifold += 1     
print(scores)
print(np.mean(scores), np.std(scores))

model_name = 'ridge_st'

np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

fold 0
0 0.708340402318
prepare test
[ 0.11833642  0.06954676  0.04558661 ...,  0.05827532  0.05132899
  0.02449793]
fold 1
1 0.688346706663
prepare test
[ 0.12280386  0.07025359  0.04505599 ...,  0.05958592  0.05212693
  0.02435883]
fold 2
2 0.684819165494
prepare test
[ 0.12269685  0.06897689  0.04413718 ...,  0.05876828  0.05143197
  0.02353197]
fold 3
3 0.701067555645
prepare test
[ 0.12503102  0.06917574  0.04570564 ...,  0.05932545  0.05168608
  0.02553804]
fold 4
4 0.691026574999
prepare test
[ 0.12058299  0.06864695  0.04472805 ...,  0.05722517  0.04967278
  0.02260276]
fold 5
5 0.693334580204
prepare test
[ 0.12662066  0.06941002  0.04534232 ...,  0.05896353  0.05452349
  0.02575778]
fold 6
6 0.690914034276
prepare test
[ 0.12218287  0.06798539  0.04394661 ...,  0.05787704  0.05201676
  0.02397596]
fold 7
7 0.705269152806
prepare test
[ 0.1224833   0.06946139  0.04605142 ...,  0.05979037  0.05210769
  0.0248025 ]
fold 8
8 0.695439919706
prepare test
[ 0.12474733  0.07133369  0

,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.322249
1,ac4b8244f3ae82df511b002257473c11,0.094247
2,483d8b91e49522c8a5bbe37f3872c749,0.166798
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.073315
4,fdbfba9842ff0bf86d600eb334c7c42b,0.075228


In [41]:
model_name = 'extra_trees_st'
import xgboost as xgb
import scipy.sparse as sp
from sklearn.feature_selection import SelectPercentile
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import ExtraTreesClassifier

def make_agg_features(X, train_index, test_index, test_data):
    return X.loc[train_index,:], X.loc[test_index,:], test_data
    
train_cols = ['records', 'max_days', 'min_days', 
              'nnet10','nnet11','xgb_single','nnet12','nnet13','lgbm1','dt2','dt1','dt3',
              'nnet3','nnet1','nnet14','nnet15','timp','nnet19','nnet20','nnet21','nnet22',
              'ftrl','vw2','sgd1','nnet16','nnet17','nnet18','tfidf2','tfidf1','deeptree',
              'vw','ftrl_50',
              'nnet4','nnet5','nnet6','nnet7',
              'nnet8','lgbmb'] + ['svd_title_'+str(i+1) for i in range(3)] + ['asvd_'+str(i+1) for i in range(3)]

kf = KFold(n_splits=10, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):
    print('fold', ifold)
       
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va,X_te = make_agg_features(X,train_index,test_index,x_te)
    X_tr = X_tr[train_cols].fillna(0).values
    X_va = X_va[train_cols].fillna(0).values
    X_te = X_te[train_cols].fillna(0).values
    
    #scaler = MinMaxScaler(feature_range=(0, 1))
    
    #X_tr = scaler.fit_transform(X_tr)
    #X_va = scaler.transform(X_va)
    #X_te = scaler.transform(X_te)
    
    model = ExtraTreesClassifier(n_estimators=110, max_depth=4, min_samples_leaf=10)
    model.fit(X_tr,y_tr)
    yhat = model.predict_proba(X_va)[:,1]
    scores.append(roc_auc_score(y_va,yhat))
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    
    ytst = model.predict_proba(X_te)[:,1]
    print(ytst)
    y_pred += minmax_scale(ytst)*0.1
    
    del X_tr,X_va,X_te
    gc.collect()    
    
    ifold += 1     
print(scores)
print(np.mean(scores), np.std(scores))

np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

fold 0
0 0.705511680265
prepare test
[ 0.08642063  0.07267376  0.04741563 ...,  0.05219046  0.04414738
  0.02277783]
fold 1
1 0.685630363913
prepare test
[ 0.0889254   0.07548698  0.0477886  ...,  0.05348164  0.04364017
  0.02180306]
fold 2
2 0.67987137222
prepare test
[ 0.09477167  0.07239947  0.04755529 ...,  0.05354144  0.04526149
  0.02388841]
fold 3
3 0.697525071504
prepare test
[ 0.0971157   0.07807275  0.04711992 ...,  0.05378634  0.0472256
  0.02222092]
fold 4
4 0.689512048692
prepare test
[ 0.09449069  0.07382289  0.04677723 ...,  0.05336513  0.04462568
  0.02210347]
fold 5
5 0.692438852205
prepare test
[ 0.08735287  0.07449404  0.04748723 ...,  0.05279011  0.04398263
  0.02288745]
fold 6
6 0.686541830548
prepare test
[ 0.09340419  0.07275115  0.04686383 ...,  0.05253238  0.04410074
  0.02276739]
fold 7
7 0.704164519576
prepare test
[ 0.0927281   0.0763958   0.04800999 ...,  0.05401471  0.04582339
  0.02258885]
fold 8
8 0.691851131956
prepare test
[ 0.09485311  0.07455373  0.0

,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.325353
1,ac4b8244f3ae82df511b002257473c11,0.094868
2,483d8b91e49522c8a5bbe37f3872c749,0.170970
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.054060
4,fdbfba9842ff0bf86d600eb334c7c42b,0.041742


In [42]:
model_name = 'rf_trees_st'
import xgboost as xgb
import scipy.sparse as sp
from sklearn.feature_selection import SelectPercentile
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier

def make_agg_features(X, train_index, test_index, test_data):
    return X.loc[train_index,:], X.loc[test_index,:], test_data
    
train_cols = ['records', 'max_days', 'min_days', 
              'nnet10','nnet11','xgb_single','nnet12','nnet13','lgbm1','dt2','dt1','dt3',
              'nnet3','nnet1','nnet14','nnet15','timp','nnet19','nnet20','nnet21','nnet22',
              'ftrl','vw2','sgd1','nnet16','nnet17','nnet18','tfidf2','tfidf1','deeptree',
              'vw','ftrl_50',
              'nnet4','nnet5','nnet6','nnet7',
              'nnet8','lgbmb'] + ['svd_title_'+str(i+1) for i in range(3)] + ['asvd_'+str(i+1) for i in range(3)]

kf = KFold(n_splits=10, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):
    print('fold', ifold)
       
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va,X_te = make_agg_features(X,train_index,test_index,x_te)
    X_tr = X_tr[train_cols].fillna(0).values
    X_va = X_va[train_cols].fillna(0).values
    X_te = X_te[train_cols].fillna(0).values
    
    #scaler = MinMaxScaler(feature_range=(0, 1))
    
    #X_tr = scaler.fit_transform(X_tr)
    #X_va = scaler.transform(X_va)
    #X_te = scaler.transform(X_te)
    
    model = RandomForestClassifier(n_estimators=110, max_depth=4, min_samples_leaf=10)
    model.fit(X_tr,y_tr)
    yhat = model.predict_proba(X_va)[:,1]
    scores.append(roc_auc_score(y_va,yhat))
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    
    ytst = model.predict_proba(X_te)[:,1]
    print(ytst)
    y_pred += minmax_scale(ytst)*0.1
    
    del X_tr,X_va,X_te
    gc.collect()    
    
    ifold += 1     
print(scores)
print(np.mean(scores), np.std(scores))

np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

fold 0
0 0.70804775327
prepare test
[ 0.116225    0.07927322  0.05347689 ...,  0.05864745  0.05113927
  0.0223893 ]
fold 1
1 0.688951905744
prepare test
[ 0.12104915  0.08337226  0.05611021 ...,  0.05844343  0.05300218
  0.02179187]
fold 2
2 0.682984219056
prepare test
[ 0.11450288  0.08221126  0.05529802 ...,  0.05637302  0.0518993
  0.02136322]
fold 3
3 0.700850791707
prepare test
[ 0.12060298  0.08042467  0.05673899 ...,  0.05704012  0.053041    0.0229039 ]
fold 4
4 0.692135722238
prepare test
[ 0.12076868  0.07826234  0.05648222 ...,  0.055417    0.05190627
  0.02325941]
fold 5
5 0.694826201798
prepare test
[ 0.12203647  0.08352681  0.05279225 ...,  0.05720413  0.05353128
  0.02144274]
fold 6
6 0.689444576416
prepare test
[ 0.11810649  0.08007047  0.05446343 ...,  0.05510314  0.05291391  0.02166   ]
fold 7
7 0.705098681343
prepare test
[ 0.12421809  0.08521453  0.05282894 ...,  0.05790903  0.05262791
  0.02173341]
fold 8
8 0.69417389921
prepare test
[ 0.1213723   0.08193032  0.0552

,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.407703
1,ac4b8244f3ae82df511b002257473c11,0.096418
2,483d8b91e49522c8a5bbe37f3872c749,0.201525
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.055439
4,fdbfba9842ff0bf86d600eb334c7c42b,0.056374


In [43]:
meta = [
              'nnet10','nnet11','xgb_single','nnet12','nnet13','lgbm1','dt2','dt1','dt3',
              'nnet3','nnet1','nnet14','nnet15','timp','nnet19','nnet20','nnet21','nnet22',
              'ftrl','vw2','sgd1','nnet16','nnet17','nnet18','tfidf2','tfidf1','deeptree',
              'vw','ftrl_50',
              'nnet4','nnet5','nnet6','nnet7',
              'nnet8','lgbmb']
for m in ['ridge_st','extra_trees_st','rf_trees_st','xgb_e7','lgb_e9']:
    sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
    sample_sub['target'] = minmax_scale(np.load(results_dir + 'test_' + m + '.npy'))
    sample_sub.columns = ['uid',m]
    x_te = x_te.merge(sample_sub, on='uid', how='left')
    X[m] = minmax_scale(np.load(results_dir + 'train_'+m+'.npy'))
    meta.append(m)

In [47]:
model_name = 'lgb_e10'
import xgboost as xgb
import scipy.sparse as sp
from sklearn.feature_selection import SelectPercentile
def save_submit(model_name, folds, y_pred):
    global x_te
    sub = x_te[['uid','target']].copy()
    sub['target'] = y_pred
    sub.columns = ['cuid','target']
    sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')
    sample_sub = sample_sub.merge(sub, on='cuid', how='left')
    sample_sub[['target']].to_csv(results_dir + model_name + '_' + str(folds) + 'folds.csv', header=False, index=False)
    del sub,sample_sub
    gc.collect()
    
def mean_encode_test(df, y, test,k,column):
    mean_0 = np.zeros((test.shape[0],1))
    df['target'] = y
    m0 = np.mean(y)  
    y0s = df[['target',column]].groupby(column).agg(np.mean).reset_index()
    y0s.columns = [column,'target_mean']
    vc = df[column].value_counts().reset_index()
    vc.columns = [column,'counts']
    test = test.merge(y0s, on = column,how= 'left').merge(vc, on = column,how= 'left')
    test['mean_target'] = (test.target_mean * test.counts + k * m0)/(test.counts + k)
    mean_0 = np.array(test['mean_target']).reshape(-1,1)
    return mean_0    

def mean_encode_self(df, y, kf, k, column):
    mean_0 = np.zeros((y.shape[0],1))
    df['target'] = y
    m0 = np.mean(y)
    for dev_index, val_index in kf: 
        dev_X, val_X = df.iloc[dev_index,:], df.iloc[val_index,:]
        y0s = dev_X[['target',column]].groupby(column).agg(np.mean).reset_index()
        y0s.columns = [column,'target_mean']
        vc = dev_X[column].value_counts().reset_index()
        vc.columns = [column,'counts']
        val_X = val_X.merge(y0s, on = column,how= 'left').merge(vc, on = column,how= 'left')
        val_X['mean_target'] = (val_X.target_mean * val_X.counts + k * m0)/(val_X.counts + k)
        mean_0[val_index,:] = np.array(val_X['mean_target']).reshape(-1,1)       
    return mean_0

def make_agg_features(X, train_index, test_index, test_data):
    te_cols = ['most_freq_cat']
    kf = KFold(n_splits = 5, random_state=2018, shuffle=True)
    for c in te_cols:
        X.loc[test_index,c + '_te'] = mean_encode_test(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), X.loc[test_index,:].copy(), 10.0, c)
        test_data.loc[:,c + '_te'] = mean_encode_test(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), test_data.copy(), 10.0, c)
        X.loc[train_index,c + '_te'] = mean_encode_self(X.loc[train_index,:].copy(), X.loc[train_index,'target'].copy(), kf.split(X.loc[train_index,:]), 10.0, c)
    return X.loc[train_index,:], X.loc[test_index,:], test_data
    
train_cols = ['sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
              'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
              'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
              'num_times_cat_eq_0', 'num_times_cat_eq_1', 'num_times_cat_eq_2',
              'num_times_cat_eq_3', 'num_times_cat_eq_4', 'num_times_cat_eq_5',
              'records', 'max_days', 'min_days', 'sum_values_f1_max',
              'num_keys_f1_max', 'sum_values_f2_max', 'num_keys_f2_max',
              'sum_values_f3_max', 'num_keys_f3_max', 'sum_values_f1_mean',
              'num_keys_f1_mean', 'sum_values_f2_mean', 'num_keys_f2_mean',
              'sum_values_f3_mean', 'num_keys_f3_mean', 'max_day_cntr',
              'mean_day_cntr', 'nuniq_keys_f1', 'nuniq_keys_f1.1',
              'nuniq_keys_f1.2', 'sumval_keys_f1', 'sumval_keys_f1.1',
              'sumval_keys_f1.2', 'most_freq_cat_te', 'diff_num_cats', 'unique_days','max_f1','max_f2','max_f3',
              'svd_description_1','svd_description_2','svd_description_3','svd_description_4','svd_description_5',
              'svd_description_6','svd_description_7','svd_description_8','svd_description_9','svd_description_10',
              'most_freq_cat_te'] +\
             ['svd_title_'+str(i+1) for i in range(10)] +\
             ['asvd_'+str(i+1) for i in range(10)] +\
             meta[-5:]


#,'most_freq_cat_te','nnet4','nnet5','nnet6','nnet7','nnet10','nnet11','nnet8',

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 16,
    'max_depth' : 4,
    #'min_data' : 30,
    'lambda_l1' : 1.5,
    'lambda_l2' : 30.2,
    'bagging_freq' : 2,
    'bagging_fraction' : 0.8,
    'is_unbalance': True,
    'learning_rate': 0.005,
    'feature_fraction': 0.5,
    'min_data_in_leaf':100,
    'verbose': 0
}

kf = KFold(n_splits=10, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

train_mat = sp.hstack([train_mat1,train_mat2,train_mat3]).tocsr().astype(np.bool).astype(np.float32)
test_mat = sp.hstack([test_mat1,test_mat2,test_mat3]).tocsr().astype(np.bool).astype(np.float32)

for train_index,test_index in kf.split(X):
    print('fold', ifold)
       
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va,X_te = make_agg_features(X,train_index,test_index,x_te)
    X_tr = X_tr[train_cols]
    X_va = X_va[train_cols]
    X_te = X_te[train_cols]
    
    yy = y_tr
    ssp = SelectPercentile(percentile=0.1)  
    ssp.fit(train_mat[train_index], yy)
    sp_train_mat = ssp.transform(train_mat[train_index])
    sp_val_mat = ssp.transform(train_mat[test_index])
    sp_test_mat = ssp.transform(test_mat)   
    
    print('prepare train')
    X_tr = sp.hstack([     X_tr, sp_train_mat   ]).tocsr() #, train_mat_pcat[train_index] 
    #print(X_tr.shape)
    #print('prepare valid')
    X_va = sp.hstack([        X_va, sp_val_mat    ]).tocsr()     #, train_mat_pcat[test_index]
    #print('prepare test')
    X_te = sp.hstack([       X_te, sp_test_mat    ]).tocsr()      #, test_mat_pcat

    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=[tr_data,va_data],
                      num_boost_round=8000,
                      early_stopping_rounds=300,
                      verbose_eval=100)
    
    yhat = model.predict(X_va, model.best_iteration)
    scores.append(roc_auc_score(y_va,yhat))
    print(ifold,roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    
    ytst = model.predict(X_te, model.best_iteration)
    print(ytst)
    y_pred += minmax_scale(ytst)*0.1
    
    del X_tr,X_va,tr_data,va_data, sp_train_mat, sp_val_mat, sp_test_mat
    gc.collect()    
    
    ifold += 1     
print(scores)
print(np.mean(scores), np.std(scores))    

np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)
sample_sub.head()

fold 0
prepare train
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.696862	valid_1's auc: 0.707882
[200]	training's auc: 0.69746	valid_1's auc: 0.708169
[300]	training's auc: 0.697925	valid_1's auc: 0.708068
[400]	training's auc: 0.698491	valid_1's auc: 0.707993
Early stopping, best iteration is:
[176]	training's auc: 0.697309	valid_1's auc: 0.708271
0 0.70827063927
prepare test
[ 0.64121986  0.55752167  0.49632673 ...,  0.50954521  0.49885458
  0.34835354]
fold 1
prepare train
Training until validation scores don't improve for 300 rounds.
[100]	training's auc: 0.699094	valid_1's auc: 0.688708
[200]	training's auc: 0.699623	valid_1's auc: 0.688651
[300]	training's auc: 0.70016	valid_1's auc: 0.688616
Early stopping, best iteration is:
[24]	training's auc: 0.698366	valid_1's auc: 0.68896
1 0.688960350922
prepare test
[ 0.5272917   0.51023533  0.49676146 ...,  0.50329465  0.49723536
  0.47430945]
fold 2
prepare train
Training until validation score

,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.942088
1,ac4b8244f3ae82df511b002257473c11,0.511105
2,483d8b91e49522c8a5bbe37f3872c749,0.762535
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.378906
4,fdbfba9842ff0bf86d600eb334c7c42b,0.376711
